# The Super Sub: Impact of Attacking Substitutes in the Premier League (2013–2023)

## Introduction
This project investigates the impact of "Super Subs"—attacking players brought on as substitutes who score crucial goals late in matches. Specifically, we explore whether these players are more likely than expected to score goals that equalize or win games after being subbed on when their team is tied or losing.

## Hypothesis
**H₀ (Null):** Attacking substitutes do not have a significantly higher chance of scoring game-tying or winning goals compared to starters, when adjusting for playing time and match context.  
**H₁ (Alternative):** Attacking substitutes (Super Subs) have a significantly higher chance of scoring game-tying or winning goals, relative to their time on the pitch and the state of the match.

## Scope
- **League:** English Premier League  
- **Time Period:** 2013–2023 (10 seasons)  
- **Focus:** Substitutes entering after the 60th minute in tied or losing game states

## Constraints
- Publicly available data sources may limit event granularity (e.g., exact match state at time of substitution or goal).
- Player position classification may not always be consistent across datasets.

## Biases
- Stronger teams tend to have deeper benches, possibly skewing substitute impact.
- Tactical shifts or injuries may affect the context of substitutions.
- Late-game chaos and small sample size may distort scoring likelihoods.